# Using Tweepy for Fun and Profit

Tweepy is a library that lets you use Twitter via Python. Here are all the docs for using it: http://tweepy.readthedocs.org/en/v3.5.0/

Because Tweepy is not part of the default Anaconda distribution we need to install is separately. In a terminal type the following: **`pip install tweepy`** and that should download and install the library so that you can use it. If everything worked, the next cell should execute with no errors.


In [ ]:
import tweepy

Before we can really DO anything with Tweepy we need to authenticate it. Basically we need to log into Twitter with the right credentials. Below I use credentials to use the app I've created for the @cjclassbot twitter account. To create your own account and authenticate your own bot continue with the following instructions.

**Create a Twitter Account**

The first step to doing this is to create a Twitter account that you can use. Unless you want to pollute your own Twitter timeline (if you already use it) then you'll want to create a new account. A helpful tip from Dan Nguyen at Stanford is that it's easiest to register the new Twitter account with the email address of yournormalname+mytwitterbot@gmail.com. GMail will redirect all of those emails to your normal gmail account but Twitter things its a unique email address. 

You will also have to add a phone number to the account which is verified via text message in order to proceed to the next step. 

** Create a Twitter App**

Follow the instructions here: [http://www.compjour.org/tutorials/getting-started-with-tweepy/](http://www.compjour.org/tutorials/getting-started-with-tweepy/) to create a Twitter app. We need to do this in order to get the necessary authentication tokens to log in to Twitter programmatically. 

You'll get four tokens that you need to copy into the code below. In general you should NOT publish these keys. I am only publishing the keys for the @cjclassbot account to make demonstrating this a bit easier, and knowing that I won't use this account for any other purpose.  

In [ ]:
CONSUMER_KEY = ""
CONSUMER_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

Now that we've authenticated we can use tweepy to make calls to the Twitter API. I can print out the latest 20 tweets from a given account by doing the following:

In [ ]:
tweets = api.user_timeline("merrillcollege")
for tweet in tweets:
    print tweet.created_at, " ", tweet.text
    print "\n"

There's lots we can do with Tweepy. The complete API documentation is here: [http://tweepy.readthedocs.org/en/v3.5.0/api.html](http://tweepy.readthedocs.org/en/v3.5.0/api.html). But it still can't do *everything* so there are some limitations. At times it's necesary to go back to the Twitter documentation itself [https://dev.twitter.com/rest/public](https://dev.twitter.com/rest/public), and there are important details to be aware of such as rate limits which affect how often an account can tweet or how many API calls it can make in a given window of time.

**Sending a tweet**

In [ ]:
# update_status function, if we add a in_reply_to_status_id parameter we can make the bot reply to another tweet. 
new_tweet = api.update_status("Hi, this is a new tweet!")

In [ ]:
# Tweepy returns an object with metadata for the tweet we just sent. 
new_tweet

**Get User Details**

In [ ]:
import json
import pprint

# get_user
user = api.get_user(screen_name="merrillcollege")

# We can grab the json version of the object and print it 
pprint.pprint(user._json)

In [ ]:
# And we can then access individual attributes of the user
print "Followers:",user.followers_count
print "Friends:",user.friends_count

**List Timeline**

Can be useful if you want your bot just to pay attention to a few specific accounts.

In [ ]:
# list_timeline
# 1st parameter is the list owner, and 2nd is the slug
# If you're on twitter the URL looks like: https://twitter.com/YuriEngelhardt/lists/dagstuhl-datadriven-story
list_tweets = api.list_timeline("YuriEngelhardt", "dagstuhl-datadriven-story", per_page=10, page=0)
for tweet in list_tweets:
    print tweet.created_at, " ", tweet.text
    print "\n"

In [ ]:
# trends_closest
# Look up lat, long coordinates on google maps by right-clicking
latitude = 38.987004
longitude =  -76.948303
nearby_locations_with_trends = api.trends_closest(latitude, longitude)
# This gives us the nearby locations that have trending topics associated with them
print nearby_locations_with_trends[0]["woeid"]

# now to get the actual trends for washington
trends = api.trends_place(nearby_locations_with_trends[0]["woeid"])
pprint.pprint(trends)

In [ ]:
for t in trends[0]["trends"]:
    print t["name"]

**Search**

Let's search for the results of the first trend we found. You should be able to get up to 1000 results via the `count` parameter. There are ways to get more than that but you need to use the `since_id` parameter and do some custom code to keep track of ids.

In [ ]:
# search
trend = trends[0]["trends"][0]
print trend["name"]

# Get the first page of 10 results
search_result = api.search(trend["name"], lang="en", count=1000)
for tweet in search_result:
    print tweet.created_at, " ", tweet.text
    print "\n"

------

**Book Bot**  
An activity if there's time: What if anything could we tweet that's interesting about any of the books on the NYT non-fiction list?

In [2]:
import requests, json
import pandas as pd
url = 'http://api.nytimes.com/svc/books/v3/lists/hardcover-nonfiction.json?api-key=sample-key' 
api_response = requests.get(url).json()
books_data = pd.read_json(json.dumps(api_response["results"]["books"]))
books_data

,age_group,amazon_product_url,article_chapter_link,asterisk,author,book_image,book_review_link,contributor,contributor_note,dagger,...,isbns,price,primary_isbn10,primary_isbn13,publisher,rank,rank_last_week,sunday_review_link,title,weeks_on_list
0,,http://www.amazon.com/When-Breath-Becomes-Paul...,,0,Paul Kalanithi,http://du.ec2.nytimes.com.s3.amazonaws.com/prd...,,by Paul Kalanithi,,0,...,"[{u'isbn10': u'081298840X', u'isbn13': u'97808...",0,081298840X,9780812988406,Random House,1,1,,WHEN BREATH BECOMES AIR,5
1,,http://www.amazon.com/The-Name-God-Is-Mercy-eb...,,0,Pope Francis with Andrea Tornielli,http://du.ec2.nytimes.com.s3.amazonaws.com/prd...,,by Pope Francis with Andrea Tornielli,,0,...,"[{u'isbn10': u'0399588639', u'isbn13': u'97803...",0,0399588639,9780399588631,Random House,2,4,,THE NAME OF GOD IS MERCY,5
2,,http://www.amazon.com/Between-World-Me-Ta-Nehi...,,0,Ta-Nehisi Coates,http://du.ec2.nytimes.com.s3.amazonaws.com/prd...,http://www.nytimes.com/2015/08/17/books/review...,by Ta-Nehisi Coates,,0,...,"[{u'isbn10': u'0812993543', u'isbn13': u'97808...",0,0812993543,9780812993547,Spiegel & Grau,3,3,,BETWEEN THE WORLD AND ME,31
3,,http://www.amazon.com/The-Road-Little-Dribblin...,,0,Bill Bryson,http://du.ec2.nytimes.com.s3.amazonaws.com/prd...,,by Bill Bryson,,0,...,"[{u'isbn10': u'0385539282', u'isbn13': u'97803...",0,0385539282,9780385539289,Doubleday,4,5,http://www.nytimes.com/2016/01/31/books/review...,THE ROAD TO LITTLE DRIBBLING,4
4,,http://www.amazon.com/Turning-Tables-Housewife...,,0,Teresa Giudice and K C Baker,http://du.ec2.nytimes.com.s3.amazonaws.com/prd...,,by Teresa Giudice and K. C. Baker,,0,...,"[{u'isbn10': u'1501135104', u'isbn13': u'97815...",0,1501135104,9781501135101,Gallery Books,5,0,,TURNING THE TABLES,1
5,,http://www.amazon.com/Dark-Money-History-Billi...,,0,Jane Mayer,http://du.ec2.nytimes.com.s3.amazonaws.com/prd...,,by Jane Mayer,,0,...,"[{u'isbn10': u'0385535597', u'isbn13': u'97803...",0,0385535597,9780385535595,Doubleday,6,6,http://www.nytimes.com/2016/01/24/books/review...,DARK MONEY,4
6,,http://www.amazon.com/Originals-How-Non-Confor...,,0,Adam Grant,http://du.ec2.nytimes.com.s3.amazonaws.com/prd...,,by Adam Grant,,0,...,"[{u'isbn10': u'0525429565', u'isbn13': u'97805...",0,0525429565,9780525429562,Viking,7,2,,ORIGINALS,2
7,,http://www.amazon.com/In-Other-Words-Jhumpa-La...,,0,Jhumpa Lahiri and Ann Goldstein,http://du.ec2.nytimes.com.s3.amazonaws.com/prd...,,by Jhumpa Lahiri and Ann Goldstein,,0,...,"[{u'isbn10': u'1101875550', u'isbn13': u'97811...",0,1101875550,9781101875551,Knopf,8,0,,IN OTHER WORDS,1
8,,http://www.amazon.com/Killing-Reagan-Violent-A...,,0,Bill O'Reilly and Martin Dugard,http://du.ec2.nytimes.com.s3.amazonaws.com/prd...,,by Bill O'Reilly and Martin Dugard,,0,...,"[{u'isbn10': u'1627792414', u'isbn13': u'97816...",0,1627792414,9781627792417,Holt,9,8,,KILLING REAGAN,21
9,,http://www.amazon.com/Thomas-Jefferson-Tripoli...,,0,Brian Kilmeade and Don Yaeger,http://du.ec2.nytimes.com.s3.amazonaws.com/prd...,,by Brian Kilmeade and Don Yaeger,,0,...,"[{u'isbn10': u'1591848067', u'isbn13': u'97815...",0,1591848067,9781591848066,Sentinel,10,9,,THOMAS JEFFERSON AND THE TRIPOLI PIRATES,15
